# DPO‑Finance Quick‑Start Notebook
This notebook walks through an end‑to‑end example:
1. Price data download (2 symbols, 3 years)
2. Preference‑pair generation
3. Model training (few epochs for demo)
4. Evaluation + simple back‑test

> Note: For a full‑scale experiment, use the CLI scripts and full dataset instead.

In [ ]:
# Install dependencies in this Colab/local session (skip if env already set)
!pip -q install yfinance pytorch-lightning hydra-core wandb ta-lib pandas numpy scikit-learn finta

In [ ]:
import os, pathlib, json, torch, pandas as pd, numpy as np
from importlib import reload

# Clone or mount repo (replace with your path)
REPO_DIR = pathlib.Path('dpo_financial_repo').resolve()
if not REPO_DIR.exists():
    !git clone https://github.com/yourname/dpo_financial_repo.git
    REPO_DIR = pathlib.Path('dpo_financial_repo').resolve()
import sys, importlib; sys.path.append(str(REPO_DIR/'src'))

In [ ]:
# 1. Download price data (AAPL & MSFT, 2022‑2024)
from dpo_forecasting.data.download import fetch_symbol, save_data
raw_dir = REPO_DIR/'data/raw'
raw_dir.mkdir(parents=True, exist_ok=True)
for sym in ['AAPL', 'MSFT']:
    df = fetch_symbol(sym, '2022-01-01', '2024-12-31')
    save_data(df, raw_dir/f'{sym}.csv')
print('Done.')

In [ ]:
# 2. Make preference pairs (5‑day horizon, 80/20 quantiles)
from dpo_forecasting.data.make_pairs import make_pairs_for_symbol, load_price_csv
pairs = []
for csv in raw_dir.glob('*.csv'):
    sym = csv.stem
    df = load_price_csv(csv)
    p = make_pairs_for_symbol(df, lookahead=5, gq=0.8, bq=0.2)
    p.insert(0, 'symbol', sym)
    pairs.append(p)
pairs_df = pd.concat(pairs, ignore_index=True)
pairs_path = REPO_DIR/'data/demo_pairs.parquet'
pairs_df.to_parquet(pairs_path, index=False)
pairs_df.head()

In [ ]:
# 3. Train tiny model (3 epochs)
from dpo_forecasting.data.dataset import PreferenceDataModule
from dpo_forecasting.models.dpo_model import DPOModel
import pytorch_lightning as pl

cfg = {
    'model': {'input_dim': 29, 'hidden_sizes': [64, 64]},
    'train': {'lr': 1e-3, 'kl_coeff': 0.0},
}
dm = PreferenceDataModule(pairs_file=str(pairs_path), prices_dir=str(raw_dir), lookback=30, batch_size=128, val_fraction=0.1)
model = DPOModel(cfg)
trainer = pl.Trainer(max_epochs=3, logger=False, enable_checkpointing=False, deterministic=True)
trainer.fit(model, datamodule=dm)

In [ ]:
# 4. Evaluate pairwise accuracy
from torch.utils.data import DataLoader
ds = dm.val_ds if dm.val_ds is not None else dm.train_ds
dl = DataLoader(ds, batch_size=512)
acc = 0
total = 0
for b in dl:
    acc += (model(b['good']) > model(b['bad'])).sum().item()
    total += len(b['good'])
print('Directional accuracy:', acc/total)

In [ ]:
# 5. Quick back‑test (no costs)
from dpo_forecasting.data.dataset import ReturnWindowExtractor
extractor = ReturnWindowExtractor(30)
closes = load_price_csv(raw_dir/'AAPL.csv')
# compute equity curve for illustration... (simplified)
scores = []
for i in range(30, len(closes)-1):
    feat = extractor(closes, i)
    scores.append(model(torch.tensor(feat).unsqueeze(0)).item())
scores = np.array(scores)
high = np.quantile(scores, 0.75); low = np.quantile(scores, 0.25)
pos = np.where(scores>high, 1, np.where(scores<low, -1, 0))
ret = pos * (closes[31:] - closes[30:-1]) / closes[30:-1]
equity = np.cumprod(1+ret)
print('Mini Sharpe:', (ret.mean()*252)/(ret.std()*np.sqrt(252)))